In [13]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
import transformers
import torch
import pandas as pd
from tqdm import tqdm

In [15]:
# 데이터 불러오기
df = pd.read_csv("../data/video_script.csv", encoding="utf-8-sig", index_col=0)
df.head()

,video_id,video_title,video_script
0,RsVKZrt2DSo,대리모 사건이 소환한 합법화 논쟁 / KBC뉴스,리미나 난인 부부가 브로커에게 돈을 주고 출산을 의뢰한 대리모 범죄가 14년 만에 ...
1,2CpgC1K7jNI,대포죽순이요????푸바오 핸펀몰카사건? 10.6 푸바오 이야기 FUBAO PANDA...,바 사님이 부르시는데 핸드폰으로 후버 찍어 주시는데 부바 부르는 목소리에 벌떡 일어...
2,C-jOF1sgdOg,"일본에 남겨진 한국의 문화유산【아시아의 숨은 혼, 백제를 가다 Part 2】",고대 일본의 수도였던 오사가 동에 위치한 가시와라시 5세기 후반에 조성된 대형 무덤...
3,giUtBrfvLJU,김동건변호사 고향사찰 구천면 정수사 다녀갑니다,넘게 있어요 이거 위가요 쭉 있는데 있는 바 예 그래고 그냥 이렇게 해라 너무 많이...
4,8HRFrskGuks,"""롤드컵의 페이커는 다릅니다"" 고전파 모드 페이커 사일러스 등장 | 롤 하이라이트 ...",리시드 비지를 상대로 엄청난 활약을 보여준 페이커의 사일러스가 등장했습니다 대회 이...


In [16]:
torch.cuda.empty_cache()  # GPU 메모리 해제

In [17]:
# kobert 모델 불러오기
model_id = "EbanLee/kobart-summary-v3"
cache_dir = "../models"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_id, cache_dir=cache_dir)
model = BartForConditionalGeneration.from_pretrained(model_id, cache_dir=cache_dir)
# GPU로 모델을 옮김
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [18]:
df["bert_summary"] = None

# 각 스크립트에 대해 요약 생성
for row in tqdm(df.iterrows(), total=len(df)):
    script = row[1].video_script
    # 스크립트를 토크나이저를 통해 텐서로 변환, 패딩 및 잘림 처리
    inputs = tokenizer(script, return_tensors="pt", padding="max_length", truncation=True, max_length=1026).to(device)
    
    # 요약 텍스트 ID 생성
    summary_text_ids = model.generate(
        input_ids=inputs['input_ids'],  # 입력 ID
        attention_mask=inputs['attention_mask'],  # 주의 마스크
        bos_token_id=model.config.bos_token_id,  # 시작 토큰 ID
        eos_token_id=model.config.eos_token_id,  # 종료 토큰 ID
        length_penalty=1.0,  # 길이에 대한 패널티 설정
        max_length=512,  # 생성될 최대 길이
        min_length=256,  # 생성될 최소 길이
        num_beams=4,  # 빔 탐색의 수
        repetition_penalty=2.0,  # 반복되는 단어에 대한 패널티
        no_repeat_ngram_size=4,  # 반복되는 n-gram 크기
        top_k=50,  # top-k 샘플링 설정
        top_p=0.95  # top-p (nucleus) 샘플링 설정
    )

    # 생성된 요약 텍스트 ID를 디코딩하여 최종 요약 텍스트로 변환
    bert_summary = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)
    
    # 생성된 요약을 데이터프레임에 저장
    df.at[row[0], "bert_summary"] = bert_summary

df.head()

  0%|          | 0/49 [00:00<?, ?it/s]c:\Users\csu52\anaconda3\envs\csu\lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 49/49 [04:34<00:00,  5.60s/it]


,video_id,video_title,video_script,bert_summary
0,RsVKZrt2DSo,대리모 사건이 소환한 합법화 논쟁 / KBC뉴스,리미나 난인 부부가 브로커에게 돈을 주고 출산을 의뢰한 대리모 범죄가 14년 만에 ...,리미나 난인 부부가 브로커에게 돈을 주고 출산을 의뢰한 대리모 범죄가 14년 만에 ...
1,2CpgC1K7jNI,대포죽순이요????푸바오 핸펀몰카사건? 10.6 푸바오 이야기 FUBAO PANDA...,바 사님이 부르시는데 핸드폰으로 후버 찍어 주시는데 부바 부르는 목소리에 벌떡 일어...,푸바오 먹을 죽순을 한 손 가득 가지고 나오셨네요 이렇게 많은 육수는 처음 보는 거...
2,C-jOF1sgdOg,"일본에 남겨진 한국의 문화유산【아시아의 숨은 혼, 백제를 가다 Part 2】",고대 일본의 수도였던 오사가 동에 위치한 가시와라시 5세기 후반에 조성된 대형 무덤...,야고는 고대일본 장 문화에도 변화를 가져옵니다. 먼저 무덤양식 수식에서 시신 주변을...
3,giUtBrfvLJU,김동건변호사 고향사찰 구천면 정수사 다녀갑니다,넘게 있어요 이거 위가요 쭉 있는데 있는 바 예 그래고 그냥 이렇게 해라 너무 많이...,18세기 중 행다 화 구충은 기둥이 그대로 되어있고 시이 걸 그쪽에서 만나기로 했는...
4,8HRFrskGuks,"""롤드컵의 페이커는 다릅니다"" 고전파 모드 페이커 사일러스 등장 | 롤 하이라이트 ...",리시드 비지를 상대로 엄청난 활약을 보여준 페이커의 사일러스가 등장했습니다 대회 이...,리시드 비지를 상대로 엄청난 활약을 보여준 페이커의 사일러스가 등장했습니다. 3면 ...


In [19]:
torch.cuda.empty_cache()  # GPU 메모리 해제

In [20]:
# llama 모델 불러오기
model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
cache_dir = "../models"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16, "cache_dir": cache_dir},
    device_map="auto",
)
pipeline.model.eval()

c:\Users\csu52\anaconda3\envs\csu\lib\site-packages\accelerate\utils\modeling.py:1390: UserWarning: Current model requires 4224 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 36.82it/s]
Some parameters are on the meta device because they were offloaded to the disk and cpu.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [21]:
# 비디오 제목과 스크립트를 기반으로 유튜브 비디오의 내용을 설명하는 프롬프트입니다.
# - 비디오가 무엇인지 설명합니다.
# - 내용을 서론, 본론, 결론으로 나누어 한국어로 작성합니다.
# - 스크립트에 포함되지 않은 내용은 요약에 포함하지 않습니다.
PROMPT = '''
            Based on the YouTube video title and script below, 
            please explain what video it is and provide it in Korean by dividing it into introduction, body, and conclusion. 
            Do not include anything that is not in the script in the summary.
        '''


In [22]:
# llama_summary 열을 데이터프레임에 추가
df['llama_summary'] = None

# 데이터프레임의 각 행에 대해 반복
for row in tqdm(df.iterrows(), total=len(df)):
    video_title = row[1].video_title  # 비디오 제목 가져오기
    bert_summary = row[1].bert_summary  # BERT 요약 가져오기
    
    # 시스템과 사용자 메시지 설정
    messages = [
        {"role": "system", "content": f"{PROMPT}"},  # 시스템 역할로 프롬프트 설정
        {"role": "user", "content": f"""title: {video_title}, script: {bert_summary}"""}
    ]

    # 프롬프트를 토크나이저를 통해 변환
    prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False,  # 토큰화 비활성화
        add_generation_prompt=True  # 생성 프롬프트 추가
    )

    # 종료 토큰 ID 설정
    terminators = [
        pipeline.tokenizer.eos_token_id,  # EOS 토큰 ID
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")  # 사용자 정의 종료 토큰 ID
    ]

    # Llama 모델을 통해 출력 생성
    outputs = pipeline(
        prompt,
        max_new_tokens=500,  # 생성할 텍스트 최대 길이 설정
        eos_token_id=terminators,  # 종료 토큰 설정
        do_sample=False,  # 무작위성 제거
        temperature=0.4,  # 생성 시 창의성 감소
        top_p=0.8,  # 선택할 확률 분포 제한
        repetition_penalty=1.3  # 반복 방지 강화
    )

    # 생성된 요약을 데이터프레임에 저장
    df.at[row[0], 'llama_summary'] = outputs
df.head()

  0%|          | 0/49 [00:00<?, ?it/s]c:\Users\csu52\anaconda3\envs\csu\lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\csu52\anaconda3\envs\csu\lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
  2%|▏         | 1/49 [2:59:07<143:17:50, 10747.30s/it]


KeyboardInterrupt: 

In [ ]:
# 결과저장
df.to_csv("../data/video_summary.csv", encoding="utf-8-sig")